In [3]:
#웹페이지의 내용을 분석하여 csv 파일로 저장
# <table> 내부의 텍스트 저장
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen("http://en.wikipedia.org/wiki/Comparison_of_text_editors")
bs = BeautifulSoup(html, "html.parser")
# class가 wikitable인 태그들 중에서 첫번째 태그 선택
table = bs.find_all("table",{"class":"wikitable"})[0]
rows = table.find_all("tr")
# wt : 텍스트 쓰기 모드
csvFile = open("c:/data/csv/editors.csv", 'w', newline='', encoding='utf-8')
# csv 파일 저장 객체
writer = csv.writer(csvFile)
try:
    for row in rows:
        csvRow = []
        # td, th 태그의 내용을 리스트에 추가
        for cell in row.find_all(['td', 'th']):
            csvRow.append(cell.text.strip())
        writer.writerow(csvRow)
finally:
    print("csv로 저장되었습니다.")
    csvFile.close()


csv로 저장되었습니다.


In [4]:
import re
import datetime
import random
import pymysql
conn = pymysql.connect(host='localhost', port=3306, user='web', password='1234', db='myweb', charset='utf8')
cursor = conn.cursor()
cursor.execute("use myweb")
cursor.execute("delete from pages")


2

In [5]:
#랜덤 시드를 고정하면 랜덤값이 고정됨(재현성)
#여기서는 실행할 때마다 시드값이 바뀌므로 랜덤값이 고정되지 않음
random.seed(datetime.datetime.now())
def store(title, content):
    #따옴표 처리
    title= title.replace("'","''") #작은따옴표 1개를 2개로
    content = content.replace("'","''")
    sql ="insert into pages (title, content) values ( '%s' , '%s' )" % ( title, content )
    cursor.execute(sql)
    conn.commit()


C:\Users\tjoeun\AppData\Local\Temp/ipykernel_6544/3421763421.py:3: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(datetime.datetime.now())


In [7]:
#위키피디아의 url 수집
def getLinks(url):
    html = urlopen("http://ko.wikipedia.org"+url)
    obj = bs(html, "html.parser")
    #h1 태그의 내용
    title = obj.find("h1").text
    #id가 mw-content-text인 div 태그 내부의 p 태그의 텍스트
    content = obj.find("div", {"id":"mw-content-text"}).find("p").text
    #테이블에 저장
    store(title, content)
    #링크를 리턴함(정규표현식 사용, ^시작, $끝, * 0회 이상 반복)
    return obj.find("div", {"id":"bodyContent"}).find_all("a", href=re.compile("^(/wiki/)((?!:).)*$"))
    #함수 호출(주소창에 한글을 입력하면 코드에서는 url encoding 처리됨)
links = getLinks("/wiki/%EC%9C%A4%EB%8F%99%EC%A3%BC")
try:
    cnt = 0
    while True:
        cnt += 1
        if cnt >= 10: #10건만 처리
            break
    #url 리스트 중에서 랜덤으로 url 선택
    newArticle = links[random.randint(0, len(links)-1)].attrs["href"]
    print(newArticle)
    #해당 문서에 포함된 새로운 링크를 가져옴
    links = getLinks(newArticle)
finally:
    cursor.close()
    conn.close()
    print("완료되었습니다.")

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?